<a href="https://colab.research.google.com/github/Angeal13/NLP_Tweets_labeling/blob/main/08_Natural_Language_Processing_Fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to NLP Fundamentals in TF
NLP has the goal of deriving infromation out of nautural language(seuqences test/speech)

Another coomon term for NLP problems is sequence to sequence probelms(seq2seq)

#Imports

##Helper Fucntions

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-07-27 15:22:45--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-07-27 15:22:45 (74.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import *

##Modules


In [ ]:
import pandas as pd
from tensorflow import keras

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.optimizers import *

In [ ]:
import tensorflow_hub as hub


# Data

## Get the Data
The dataset will de Kaggels's intro to NLP(text example of Tweets label as disaster or not disaster)

link:https://www.kaggle.com/c/nlp-getting-started/

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2021-07-27 15:22:48--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2021-07-27 15:22:48 (118 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
unzip_data('nlp_getting_started.zip')

## Become on with the Data

### Visualize te Data
To visualize the data,it has to be read in first.

The better way,yet most complex wasy is to use Pandas

In [ ]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
#Shuffle the training DataFrame
train_df_shuffled=train_df.sample(frac=1,random_state=32)

In [ ]:
train_df_shuffled.head()

,id,keyword,location,text,target
2250,3221,deluged,London,Why are you deluged with low self-image? Take ...,1
7155,10252,war%20zone,NaN,Bedroom clean bathroom clean laundry done .....,0
3148,4522,emergency,"Indianapolis, IN",UPDATE: Indiana State Police reopening I-65 ne...,1
1857,2670,crush,GLOBAL,Had a minute alone with my crush??...it was an...,0
4760,6773,lightning,"Victoria, BC",Lightning causes six new fires on VancouveråÊI...,1


In [ ]:
#How doea the text Set llok like
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


##Split the Data into Training and Validation
Try to compare the Moldel's results to actual labeled inputs

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#Use train_testsplit to split trainig dat into Training and Validation Set
train_sentences,val_sentences,train_label,val_label=train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                     train_df_shuffled['target'].to_numpy(),
                                                                     test_size=0.15, #15% of Trainig data
                                                                     random_state=42)

#Convert it into Numbers
Tokenization vs Embedding:
* Tokeniztion: Straight mapping from token to numbers(can be modelled, but turns to get bigger really quickly)
* Embedding: Ricehr representation of relationship between tokes(can limit size + can be learened)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
#USe defaul Textvectorization Parameters
text_vetorizer=TextVectorization(max_tokens=None,#How many words in the Vocabulary
                                 standardize='lower_and_strip_punctuation',#Remove the Puntuation and lower case every character
                                 split='whitespace',#make each word into its own string
                                 ngrams=None,#create groups of words
                                 output_mode='int',#How to map tokens int number
                                 output_sequence_length=None,#How long is every sequence(default is None)
                                 pad_to_max_tokens=True,#Pad with zeros
                                 )

In [ ]:
#Find the AVG number of token(words) in the training tweets
round((sum([len(i.split())for i in train_sentences]))/len(train_sentences))

15

In [ ]:
#set up text vetorization variable
max_vocab_length=10000 #Max number of words to have in the vocabulary
max_length=15 #Max words per sentence

text_vetorizer=TextVectorization(max_tokens=max_vocab_length,
                                 output_mode='int',
                                 output_sequence_length=max_length)

In [ ]:
#Fit the text_vectoirizer to the training text
text_vetorizer.adapt(train_sentences)

### Create an embedding layer
Turn Tokenized text into embedding vector

To make the embeddign it is necesary to use TF embedding layer

Link:https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

Parameters:
* `input_dim`: Size of teh vocabulary
* `output_dim`: Size of th eoutput embedding vector
*`input_length`= length of sequences being passed to the embedding layer

In [ ]:
from tensorflow.keras import layers
embedding=layers.Embedding(input_dim=max_vocab_length,
                           input_length=max_length,
                           embeddings_initializer='uniform',
                           output_dim=128)

#Modeling

###Build a Fucntion to compare the metrics

In [ ]:
#Function to evaluate the metrics
from sklearn.metrics import *

def calculate_results(y_true,y_pred):
  '''
  Calculates model accuracy,precision,recall and f1 score of a binary classification model
  '''

  #Calculate model accuracy
  model_accuracy=accuracy_score(y_true,y_pred)*100

  #Calculate model Precisio,racall and f1 score,using the 'weighted average'
  model_precision,model_recall,model_f1,_=precision_recall_fscore_support(y_true,y_pred,average='weighted')
  model_results={
      'accuracy':model_accuracy,
      'precision':model_precision,
      'recall':model_recall,
      'f1_socre':model_f1
  }
  return model_results

In [ ]:
SAVE_DIR='model_logs'

##Model 0:Getting a Base Line
In this case Sklearn library will be used to create this model.

**Note** It is common to use non-Dl algorithmos as Baseline  and imporve until DL

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#Create tokinization adn modeling pipeline
model_0=Pipeline([
                  (('tfidf'),TfidfVectorizer()),#Convert Words to numbers using TFIDF
                  (('clf'),MultinomialNB()),#Model the text
])
#Fit Pipeline to teh trainin Data
model_0.fit(train_sentences,train_label)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [ ]:
#Evaluate model_0
baseline_score=model_0.score(val_sentences,val_label)
print(f'BaseLine model achives accuracy of: {baseline_score*100:.2f}%')

BaseLine model achives accuracy of: 78.28%


In [ ]:
#Make predictions
baseline_preds=model_0.predict(val_sentences)
baseline_preds[:20],val_label[:20]


(array([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1]),
 array([0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1]))

In [ ]:
#get the results
baseline_results=calculate_results(y_true=val_label,y_pred=baseline_preds)

print(baseline_results)

{'accuracy': 78.28371278458845, 'precision': 0.8009601599769112, 'recall': 0.7828371278458844, 'f1_socre': 0.7740111697028993}


##Model_1:Feed Forward

In [ ]:
#Building the model
inputs=layers.Input(shape=(1,),dtype='string') #Input is one dimentional
x= text_vetorizer(inputs) #Turn input text into numbers
x=embedding(x) #Embeddin layer
x=layers.GlobalAveragePooling1D()(x) #Condense the feature of each token to 1 vector
outputs=layers.Dense(1,activation='sigmoid')(x) #Create output layer
#Create the model
model_1=Model(inputs,outputs,name='model_1_dense')

In [ ]:
#compile and fit the model
model_1.compile(
    loss='binary_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)
model_1_history=model_1.fit(x=train_sentences,y=train_label,
                            epochs=5,validation_data=(val_sentences,val_label),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name='model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20210727-152350
Epoch 1/5
203/203 [==============================] - 4s 16ms/step - loss: 0.6162 - accuracy: 0.6840 - val_loss: 0.5341 - val_accuracy: 0.7662
Epoch 2/5
203/203 [==============================] - 3s 15ms/step - loss: 0.4483 - accuracy: 0.8173 - val_loss: 0.4650 - val_accuracy: 0.7951
Epoch 3/5
203/203 [==============================] - 3s 15ms/step - loss: 0.3501 - accuracy: 0.8589 - val_loss: 0.4491 - val_accuracy: 0.7942
Epoch 4/5
203/203 [==============================] - 3s 15ms/step - loss: 0.2854 - accuracy: 0.8892 - val_loss: 0.4531 - val_accuracy: 0.7898
Epoch 5/5
203/203 [==============================] - 3s 15ms/step - loss: 0.2363 - accuracy: 0.9099 - val_loss: 0.4719 - val_accuracy: 0.7907


In [ ]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#make predictions and evaluate
model_1_pred_probs=model_1.predict(val_sentences)
model_1_pred_probs[0]

array([0.10194194], dtype=float32)

In [ ]:
#convert model prediction probabilities to label format
model_1_pred=tf.squeeze(tf.round(model_1_pred_probs))

In [ ]:
#Calculate teh results
model_1_results=calculate_results(y_true=val_label,y_pred=model_1_pred)
import numpy as np
np.array(list(model_1_results.values()))>np.array(list(baseline_results.values()))

array([ True, False,  True,  True])

In [ ]:
model_1_results,baseline_results

({'accuracy': 79.07180385288967,
  'f1_socre': 0.789330460019858,
  'precision': 0.7904622622156405,
  'recall': 0.7907180385288967},
 {'accuracy': 78.28371278458845,
  'f1_socre': 0.7740111697028993,
  'precision': 0.8009601599769112,
  'recall': 0.7828371278458844})

## Model_2: LSTM(RNN)

The premise of a RNN  is to use teh representaiton of previous input to aid the representation of  a later input 

In [ ]:
#Build the model
inputs=layers.Input(shape=(1,),dtype='string')
x=text_vetorizer(inputs)
x=embedding(x)
#x=layers.LSTM(units=64,return_sequences=True,activation='tanh',recurrent_activation='sigmoid')(x)
#When stacking RNN cell together,there is a need to return_sequences=True
x=layers.LSTM(64)(x)
#x=layers.Dense(64,activation='relu')(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
#Create the model
model_2=Model(inputs,outputs,name='model_2_LSTM')


In [ ]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_2.compile(
    loss='binary_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)
model_2_history=model_2.fit(x=train_sentences,y=train_label,
                            epochs=5,validation_data=(val_sentences,val_label),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name='model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20210727-152420
Epoch 1/5
203/203 [==============================] - 8s 30ms/step - loss: 0.2198 - accuracy: 0.9235 - val_loss: 0.5809 - val_accuracy: 0.7767
Epoch 2/5
203/203 [==============================] - 5s 26ms/step - loss: 0.1507 - accuracy: 0.9471 - val_loss: 0.6146 - val_accuracy: 0.7750
Epoch 3/5
203/203 [==============================] - 6s 27ms/step - loss: 0.1214 - accuracy: 0.9544 - val_loss: 0.7614 - val_accuracy: 0.7636
Epoch 4/5
203/203 [==============================] - 5s 26ms/step - loss: 0.1013 - accuracy: 0.9624 - val_loss: 0.7762 - val_accuracy: 0.7715
Epoch 5/5
203/203 [==============================] - 6s 27ms/step - loss: 0.0787 - accuracy: 0.9692 - val_loss: 0.9544 - val_accuracy: 0.7618


In [ ]:
#make predicitons
model_2_pred_probs=model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[1.0828674e-03],
       [7.7813864e-04],
       [4.3240526e-01],
       [4.2182207e-04],
       [1.4689553e-01],
       [2.6369956e-01],
       [6.5642595e-04],
       [7.3448282e-01],
       [9.9185997e-01],
       [6.8518519e-04]], dtype=float32)

In [ ]:
#Convert to labels
model_2_pred=tf.squeeze(tf.round(model_1_pred_probs))
model_2_pred

<tf.Tensor: shape=(1142,), dtype=float32, numpy=array([0., 0., 1., ..., 1., 0., 0.], dtype=float32)>

In [ ]:
#calculate Model_2 results
model_2_results=calculate_results(y_true=val_label,y_pred=model_2_pred)
model_2_results,baseline_results

({'accuracy': 79.07180385288967,
  'f1_socre': 0.789330460019858,
  'precision': 0.7904622622156405,
  'recall': 0.7907180385288967},
 {'accuracy': 78.28371278458845,
  'f1_socre': 0.7740111697028993,
  'precision': 0.8009601599769112,
  'recall': 0.7828371278458844})

## Model_3: GRU RNN

In [ ]:
#Build the model
inputs=layers.Input(shape=(1,),dtype='string')
x=text_vetorizer(inputs)
x=embedding(x)
#x=layers.GRU(64,return_sequences=True)(x)
x=layers.GRU(64)(x)
outputs=layers.Dense(1,activation='sigmoid')(x)

#Create model
model_3=Model(inputs,outputs,name='model_3_GRU')

In [ ]:
model_3.compile(
    loss='binary_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)
model_3_history=model_3.fit(x=train_sentences,y=train_label,
                            epochs=5,validation_data=(val_sentences,val_label),
                            callbacks=[create_tensorboard_callback(SAVE_DIR,'model_3_GRU')])

Saving TensorBoard log files to: model_logs/model_3_GRU/20210727-152451
Epoch 1/5
203/203 [==============================] - 8s 31ms/step - loss: 0.1560 - accuracy: 0.9363 - val_loss: 0.6396 - val_accuracy: 0.7750
Epoch 2/5
203/203 [==============================] - 5s 26ms/step - loss: 0.0799 - accuracy: 0.9702 - val_loss: 0.7523 - val_accuracy: 0.7644
Epoch 3/5
203/203 [==============================] - 5s 26ms/step - loss: 0.0692 - accuracy: 0.9733 - val_loss: 0.9026 - val_accuracy: 0.7627
Epoch 4/5
203/203 [==============================] - 5s 26ms/step - loss: 0.0630 - accuracy: 0.9753 - val_loss: 1.0934 - val_accuracy: 0.7522
Epoch 5/5
203/203 [==============================] - 5s 26ms/step - loss: 0.0523 - accuracy: 0.9754 - val_loss: 1.2437 - val_accuracy: 0.7583


In [ ]:
model_3_pred_probs=model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[2.1973252e-04],
       [6.1530292e-05],
       [2.6673105e-01],
       [5.9023576e-05],
       [3.6814874e-01],
       [4.2786533e-01],
       [6.8781374e-05],
       [3.5286880e-01],
       [9.9959052e-01],
       [1.5652180e-04]], dtype=float32)

In [ ]:
model_3_pred=tf.squeeze(tf.round(model_3_pred_probs))
model_3_pred

<tf.Tensor: shape=(1142,), dtype=float32, numpy=array([0., 0., 0., ..., 1., 0., 0.], dtype=float32)>

In [ ]:
model_3_results=calculate_results(y_true=val_label,y_pred=model_3_pred)
model_3_results,baseline_results,model_2_results,model_1_results

({'accuracy': 75.83187390542906,
  'f1_socre': 0.7575645074157552,
  'precision': 0.7574890229675436,
  'recall': 0.7583187390542907},
 {'accuracy': 78.28371278458845,
  'f1_socre': 0.7740111697028993,
  'precision': 0.8009601599769112,
  'recall': 0.7828371278458844},
 {'accuracy': 79.07180385288967,
  'f1_socre': 0.789330460019858,
  'precision': 0.7904622622156405,
  'recall': 0.7907180385288967},
 {'accuracy': 79.07180385288967,
  'f1_socre': 0.789330460019858,
  'precision': 0.7904622622156405,
  'recall': 0.7907180385288967})

## Model_4: Bidirectional RNN

Normal RNNs go from Left-Right,howeve,Bidirectional RNN goes from right to Left as well as Left to Right

In [ ]:
#Build the model
inputs=layers.Input(shape=(1,),dtype='string')
x=text_vetorizer(inputs)
x=embedding(x)
x=layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x=layers.Bidirectional(layers.GRU(64))(x)
outputs=layers.Dense(1,activation='sigmoid')(x)

#create the mdoel
model_4=Model(inputs,outputs,name='model_4_Bidirectional')

In [ ]:
model_4.summary()

Model: "model_4_Bidirectional"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
bidirectional (Bidirectional (None, 15, 128)           98816     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               74496     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 1,453,441
Trainable params: 1,453,441
Non-trainable params: 0
___________________________________

In [ ]:
model_4.compile(
    loss='binary_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

model_4_history=model_4.fit(x=train_sentences,y=train_label,
                            epochs=5,validation_data=(val_sentences,val_label),
                            callbacks=[create_tensorboard_callback(SAVE_DIR,'model_4_Bidirectional')])

Saving TensorBoard log files to: model_logs/model_4_Bidirectional/20210727-152522
Epoch 1/5
203/203 [==============================] - 19s 62ms/step - loss: 0.0969 - accuracy: 0.9674 - val_loss: 0.9199 - val_accuracy: 0.7609
Epoch 2/5
203/203 [==============================] - 11s 53ms/step - loss: 0.0532 - accuracy: 0.9759 - val_loss: 1.2084 - val_accuracy: 0.7504
Epoch 3/5
203/203 [==============================] - 11s 52ms/step - loss: 0.0488 - accuracy: 0.9779 - val_loss: 1.3697 - val_accuracy: 0.7504
Epoch 4/5
203/203 [==============================] - 11s 52ms/step - loss: 0.0415 - accuracy: 0.9794 - val_loss: 1.4885 - val_accuracy: 0.7548
Epoch 5/5
203/203 [==============================] - 11s 52ms/step - loss: 0.0414 - accuracy: 0.9793 - val_loss: 1.5015 - val_accuracy: 0.7548


In [ ]:
model_4_pred_prob=model_4.predict(val_sentences)
model_4_pred=tf.squeeze(tf.round(model_4_pred_prob))
model_4_results=calculate_results(y_true=val_label,y_pred=model_4_pred)

In [ ]:
model_1_results,model_2_results,model_3_results,model_4_results

({'accuracy': 79.07180385288967,
  'f1_socre': 0.789330460019858,
  'precision': 0.7904622622156405,
  'recall': 0.7907180385288967},
 {'accuracy': 79.07180385288967,
  'f1_socre': 0.789330460019858,
  'precision': 0.7904622622156405,
  'recall': 0.7907180385288967},
 {'accuracy': 75.83187390542906,
  'f1_socre': 0.7575645074157552,
  'precision': 0.7574890229675436,
  'recall': 0.7583187390542907},
 {'accuracy': 75.4816112084063,
  'f1_socre': 0.7515835182982602,
  'precision': 0.7553049095961615,
  'recall': 0.7548161120840631})

## Model_5:1D CNN Model

In [ ]:
#Build the model
inputs=layers.Input((1,),dtype='string')
x=text_vetorizer(inputs)
x=embedding(x)
x=layers.Conv1D(filters=64,kernel_size=5,activation='relu',padding='valid',strides=1)(x)
x=layers.GlobalMaxPooling1D()(x)
outputs=layers.Dense(1,activation='sigmoid')(x)

#Create Model
model_5=Model(inputs,outputs,name='model_5_Conv_1D')

In [ ]:
model_5.compile(
    loss='binary_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)
model_5_history=model_5.fit(x=train_sentences,y=train_label,
                            epochs=5,validation_data=(val_sentences,val_label),
                            callbacks=[create_tensorboard_callback(SAVE_DIR,'model_5_conv_1D')])

Saving TensorBoard log files to: model_logs/model_5_conv_1D/20210727-160622
Epoch 1/5
203/203 [==============================] - 5s 20ms/step - loss: 0.0863 - accuracy: 0.9745 - val_loss: 1.1352 - val_accuracy: 0.7294
Epoch 2/5
203/203 [==============================] - 4s 18ms/step - loss: 0.0534 - accuracy: 0.9771 - val_loss: 1.2385 - val_accuracy: 0.7373
Epoch 3/5
203/203 [==============================] - 4s 18ms/step - loss: 0.0494 - accuracy: 0.9785 - val_loss: 1.3061 - val_accuracy: 0.7478
Epoch 4/5
203/203 [==============================] - 4s 19ms/step - loss: 0.0464 - accuracy: 0.9791 - val_loss: 1.3228 - val_accuracy: 0.7408
Epoch 5/5
203/203 [==============================] - 4s 19ms/step - loss: 0.0446 - accuracy: 0.9790 - val_loss: 1.3521 - val_accuracy: 0.7347


In [ ]:
model_5.summary()

Model: "model_5_Conv_1D"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 11, 64)            41024     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 1,321,089
Trainable params: 1,321,089
Non-trainable params: 0
_________________________________________

In [ ]:
model_5_pred_probs=model_5.predict(val_sentences)
model_5_pred=tf.squeeze(tf.round(model_5_pred_probs))
model_5_results=calculate_results(y_true=val_label,y_pred=model_5_pred)
model_5_results,model_1_results

({'accuracy': 73.4676007005254,
  'f1_socre': 0.7339651369501731,
  'precision': 0.7337562113195908,
  'recall': 0.7346760070052539},
 {'accuracy': 79.07180385288967,
  'f1_socre': 0.789330460019858,
  'precision': 0.7904622622156405,
  'recall': 0.7907180385288967})

## Model 6: Pretrained Feature Extraction

In [ ]:
#Create keras layer that call the Featrure extraction model
sentece_encoder_layer= hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                                      input_shape=[],
                                      dtype=tf.string,
                                      trainable=False,
                                      name='USE')

In [ ]:
#Build Model(in this case Sequential)
model_6=keras.Sequential([
        sentece_encoder_layer,
        layers.Dense(64,activation='relu'),
        layers.Dense(1,activation='sigmoid',name='output')
],name='model_6_USE')

In [ ]:
model_6.compile(loss='binary_crossentropy',
                optimizer=Adam(),metrics=['accuracy'])
model_6_history=model_6.fit(x=train_sentences,y=train_label,
                            epochs=5,validation_data=(val_sentences,val_label),
                            callbacks=[create_tensorboard_callback(SAVE_DIR,'model_6_USE_V2')])

Saving TensorBoard log files to: model_logs/model_6_USE_V2/20210727-165512
Epoch 1/5
203/203 [==============================] - 4s 15ms/step - loss: 0.5112 - accuracy: 0.7784 - val_loss: 0.4497 - val_accuracy: 0.8091
Epoch 2/5
203/203 [==============================] - 2s 11ms/step - loss: 0.4144 - accuracy: 0.8139 - val_loss: 0.4367 - val_accuracy: 0.8074
Epoch 3/5
203/203 [==============================] - 2s 10ms/step - loss: 0.4000 - accuracy: 0.8209 - val_loss: 0.4336 - val_accuracy: 0.8082
Epoch 4/5
203/203 [==============================] - 2s 10ms/step - loss: 0.3916 - accuracy: 0.8280 - val_loss: 0.4286 - val_accuracy: 0.8117
Epoch 5/5
203/203 [==============================] - 2s 11ms/step - loss: 0.3852 - accuracy: 0.8305 - val_loss: 0.4277 - val_accuracy: 0.8144


In [ ]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
dense_8 (Dense)              (None, 64)                32832     
_________________________________________________________________
output (Dense)               (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [ ]:
model_6_pred_prob=model_6.predict(val_sentences)
model_6_pred=tf.squeeze(tf.round(model_6_pred_prob))
model_6_results=calculate_results(y_true=val_label,y_pred=model_6_pred)
model_6_results,model_1_results

({'accuracy': 81.43607705779334,
  'f1_socre': 0.8134259219499498,
  'precision': 0.8141475346668683,
  'recall': 0.8143607705779334},
 {'accuracy': 79.07180385288967,
  'f1_socre': 0.789330460019858,
  'precision': 0.7904622622156405,
  'recall': 0.7907180385288967})